# Full Tracker: Download, Process, and Upload Data
This notebook demonstrates the full pipeline for handling raw rgb. and .csq data:
1. Download data from a cloud bucket.
2. Process the data (e.g., align videos, run detection, and tracking).
3. Upload the processed data back to the cloud bucket.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

import numpy as np

# Import Utility Functions
from collab_env.data.file_utils import expand_path, get_project_root
from collab_env.data.gcs_utils import GCSClient

# Import Custom Scripts
from collab_env.tracking.alignment_gui import align_videos, align_videos_CI
from collab_env.tracking.model.local_model_inference import infer_with_yolo
from collab_env.tracking.model.local_model_tracking import (
    generate_thermal_masks_from_bboxes,
    get_detections_from_video,
    output_tracked_bboxes_csv,
    run_tracking,
)
from collab_env.tracking.thermal_processing import (
    process_directory,
    validate_session_structure,
)
from collab_env.tracking.visualization import (  # overlay_tracks_on_video, # TODO: this doesn't work
    export_tracks_with_masks,
    plot_tracks_at_frame_bbox_from_video,
)

# Import Environment Variables

### Set flags for data processing

In [ ]:
skip_download = False
skip_thermal_extraction = False
smoke_test = "CI" in os.environ  # True if running in CI environment

### Setup gcloud

In [4]:
gcs_client = GCSClient()  # use default connection credentials

2025-09-25 12:54:12.706 | INFO     | collab_env.data.gcs_utils:__init__:36 - Using credentials from /Users/dima/git/collab-env-dashboard/config-local/collab-data-463313-c340ad86b28e.json
2025-09-25 12:54:12.708 | INFO     | collab_env.data.gcs_utils:__init__:42 - Using project collab-data-463313


Check available buckets

In [5]:
# Verify connection
print("Available buckets:", gcs_client.list_buckets())

Available buckets: ['collab-data-test', 'collab_simulated_data', 'fieldwork_curated', 'fieldwork_processed', 'fieldwork_storage_old', 'roboflow_model', 'temp-misc', 'trained_models_gnn']


Show files within buckets

In [6]:
BUCKET_NAME = "collab-data-test" if smoke_test else "fieldwork_curated"
gcs_client.glob(f"{BUCKET_NAME}/*")

['collab-data-test/test-session']

Select a session to download and process

In [7]:
# Download Data from Cloud Bucket
SESSION_FOLDER = "test-session" if smoke_test else "2023_11_26-session_0002"
CLOUD_PREFIX = f"{BUCKET_NAME}/{SESSION_FOLDER}"
gcs_client.glob(f"{CLOUD_PREFIX}/**")
LOCAL_DOWNLOAD_DIR = expand_path(f"data/raw/{SESSION_FOLDER}", get_project_root())
LOCAL_PROCESSED_DIR = expand_path(
    f"data/processed/{SESSION_FOLDER}", get_project_root()
)

Download from gcloud

In [8]:
if not skip_download or not LOCAL_DOWNLOAD_DIR.exists():
    if not LOCAL_DOWNLOAD_DIR.exists():
        LOCAL_DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)
    if not LOCAL_PROCESSED_DIR.exists():
        LOCAL_PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

    for blob in gcs_client.glob(f"{CLOUD_PREFIX}/**"):
        relative_path = Path(blob).relative_to(f"{CLOUD_PREFIX}")
        local_name = relative_path.name
        suffix = relative_path.suffix
        print(f"local_name: {local_name}, suffix: {suffix}")
        if len(str(suffix)) > 0:
            # print("File!")
            parent_folder = relative_path.parent
            if not Path(LOCAL_DOWNLOAD_DIR / parent_folder).exists():
                print(f"Creating folder: {LOCAL_DOWNLOAD_DIR / parent_folder}")
                Path(LOCAL_DOWNLOAD_DIR / parent_folder).mkdir(
                    parents=True, exist_ok=True
                )
            # print(f"parent_folder: {parent_folder}")
            local_path = LOCAL_DOWNLOAD_DIR / parent_folder / local_name
            print(f"Downloading file: {blob} to {local_path}")
            gcs_client.gcs.get_file(blob, str(local_path))
        else:

            if not Path(LOCAL_PROCESSED_DIR / relative_path).exists():
                print(f"Creating folder: {LOCAL_PROCESSED_DIR / relative_path}")
                Path(LOCAL_PROCESSED_DIR / relative_path).mkdir(
                    parents=True, exist_ok=True
                )
        # check if there is an extension, if not this is a folder and we need to create it

    # print("Downloaded files:", list(LOCAL_DOWNLOAD_DIR.iterdir()))

local_name: metadata.yaml, suffix: .yaml
local_name: rgb_1, suffix: 
local_name: rgb_1_test.MP4, suffix: .MP4
local_name: thermal_1, suffix: 
local_name: 20231126115833275.csq, suffix: .csq


Ensure everything worked properly

In [9]:
# Validate session structure
print("Validating session structure...")
issues = validate_session_structure(LOCAL_DOWNLOAD_DIR)
print(f"Issues found: {issues if len(issues)>0 else 'None'}")

Validating session structure...
Session structure issues found:
- Missing thermal_2 directory
- Missing rgb_2 directory
Issues found: ['Missing thermal_2 directory', 'Missing rgb_2 directory']


### Extract thermal video info

In [10]:
if not skip_thermal_extraction:

    # thermal files processing
    print("Processing thermal files...")

    # call with preview=False to choose the vmin/vmax automatically, otherwise the user will be asked to choose the vmin/vmax
    # process_directory(folder_path=LOCAL_DOWNLOAD_DIR, out_path=LOCAL_DOWNLOAD_DIR, color='magma', preview=True, max_frames=None, fps=30)

    if smoke_test:
        process_directory(
            folder_path=LOCAL_DOWNLOAD_DIR,
            out_path=LOCAL_PROCESSED_DIR,
            color="magma",
            preview=False,
            max_frames=1000,
            fps=30,
            vmin=3.0,
            vmax=17.0,
        )
    else:
        process_directory(
            folder_path=LOCAL_DOWNLOAD_DIR,
            out_path=LOCAL_PROCESSED_DIR,
            color="magma",
            preview=True,
            max_frames=20000,
            fps=30,
        )

Processing thermal files...
Found 1 .csq files in /Users/dima/git/collab-environment/data/raw/test-session/thermal_1

Processing [0] in thermal_1: 20231126115833275.csq
Using provided vmin=3.0, vmax=17.0


🔄 Writing frames: 100%|██████████| 1000/1000 [02:47<00:00,  5.97frame/s]


✅ Exported 1000 frames to: /Users/dima/git/collab-environment/data/processed/test-session/thermal_1/thermal_3_17.mp4
⏱ Approx. duration: 33.33 seconds at 30 fps
⚠️ Folder thermal_2 does not exist in /Users/dima/git/collab-environment/data/raw/test-session. Skipping...


#### Spatiotemporal alignment of thermal and RGB 

In [ ]:
# default parameters for alignment to run auto example
if smoke_test:
    frame_size = (640, 480)  # Default frame size
    max_frames = 1000  # Process some frames for testing
    warp_to = "rgb"  # Default warp to rgb, thermal is changing, not rgb
    rotation_angle = 181.0
    skip_homography = False  # Default to not skip homography
    skip_translation = False  # Default to not skip translation
    camera_numbers = [1]

    # precalculated parameters for example alignment
    roi = (1859, 1251, 1319, 883)  # x, y, width, height
    H = np.array(
        [
            [0.93132, -0.12753, 17.745],
            [0.066574, 0.87069, 1.573],
            [2.6001e-05, -0.00014062, 1.0003],
        ]
    )
    rgb_offset = 27
    thermal_offset = 36

    for camera in camera_numbers:
        print(f"Processing camera {camera}...")

        # Dynamically find the RGB and thermal MP4 files
        rgb_dir = LOCAL_DOWNLOAD_DIR / f"rgb_{camera}"
        if skip_thermal_extraction:
            thermal_dir = LOCAL_DOWNLOAD_DIR / f"thermal_{camera}"
        else:
            thermal_dir = LOCAL_PROCESSED_DIR / f"thermal_{camera}"

        # Find the MP4 file in the RGB directory
        rgb_video_files = list(rgb_dir.glob("*.MP4")) + list(rgb_dir.glob("*.mp4"))
        print("files in rgb_dir:", rgb_video_files)
        if len(rgb_video_files) == 0:
            print(f"No MP4 file found in {rgb_dir}. Skipping camera {camera}.")
            continue
        elif len(rgb_video_files) > 1:
            print(f"Multiple MP4 files found in {rgb_dir}. Using the first one.")
        rgb_video_path = rgb_video_files[0]

        # Find the MP4 file in the thermal directory
        thermal_video_files = list(thermal_dir.glob("*.mp4")) + list(
            thermal_dir.glob("*.MP4")
        )
        print("files in thermal_dir:", thermal_video_files)
        if len(thermal_video_files) == 0:
            print(f"No MP4 file found in {thermal_dir}. Skipping camera {camera}.")
            continue
        elif len(thermal_video_files) > 1:
            print(f"Multiple MP4 files found in {thermal_dir}. Using the first one.")
        thermal_video_path = thermal_video_files[0]

        print(f"RGB video path: {rgb_video_path}")
        print(f"Thermal video path: {thermal_video_path}")

        output_dir_rgb = LOCAL_PROCESSED_DIR / "aligned_frames" / f"rgb_{camera}"
        output_dir_thm = LOCAL_PROCESSED_DIR / "aligned_frames" / f"thermal_{camera}"
        output_dir_rgb.mkdir(parents=True, exist_ok=True)
        output_dir_thm.mkdir(parents=True, exist_ok=True)

        # Align videos
        print(f"Aligning videos for camera {camera}...")

        align_videos_CI(
            rgb_video_path,
            thermal_video_path,
            output_dir_rgb,
            output_dir_thm,
            frame_size=(640, 480),
            max_frames=1000,
            warp_to="thermal",
            rotation_angle=rotation_angle,
            skip_homography=False,
            skip_translation=True,
            crop_rect=roi,
            H=H,
            rgb_offset=rgb_offset,
            thermal_offset=thermal_offset,
        )

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Processing camera 1...
files in rgb_dir: [PosixPath('/Users/dima/git/collab-environment/data/raw/test-session/rgb_1/rgb_1_test.MP4')]
files in thermal_dir: [PosixPath('/Users/dima/git/collab-environment/data/processed/test-session/thermal_1/thermal_3_17.mp4')]
RGB video path: /Users/dima/git/collab-environment/data/raw/test-session/rgb_1/rgb_1_test.MP4
Thermal video path: /Users/dima/git/collab-environment/data/processed/test-session/thermal_1/thermal_3_17.mp4
Aligning videos for camera 1...


Cropping/Rotating video: 100%|██████████| 1000/1000 [00:19<00:00, 50.19it/s]


✅ Adjusted RGB video saved to /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames/rgb_1/cropped_rgb.mp4


Warping video: 100%|██████████| 1000/1000 [00:02<00:00, 432.96it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Warped video saved to /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames/thermal_1/warped_thermal.mp4
RGB Offset: 27, Thermal Offset: 36


Saving adjusted RGB video: 100%|██████████| 973/973 [00:01<00:00, 759.44it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


✅ Adjusted RGB video saved to /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames/rgb_1/cropped_rgb_adjusted.mp4


Saving adjusted RGB video: 100%|██████████| 964/964 [00:00<00:00, 1453.01it/s]

✅ Adjusted RGB video saved to /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames/thermal_1/warped_thermal_adjusted.mp4


This section should open a pop-up window. If you can't see it, it may have opened in the background. 

In [ ]:
# to run without CI

if not smoke_test:
    # default parameters for alignment
    frame_size = (640, 480)  # Default frame size
    max_frames = 20000  # Process many frames by default
    warp_to = "rgb"  # Default warp to rgb, thermal is changing, not rgb
    rotation_angle = 0.0  # Default rotation angle
    skip_homography = False  # Default to not skip homography
    skip_translation = False  # Default to not skip translation
    camera_numbers = [1, 2]

    for camera in camera_numbers:
        print(f"Processing camera {camera}...")

        # Dynamically find the RGB and thermal MP4 files
        rgb_dir = LOCAL_DOWNLOAD_DIR / f"rgb_{camera}"
        if skip_thermal_extraction:
            thermal_dir = LOCAL_DOWNLOAD_DIR / f"thermal_{camera}"
        else:
            thermal_dir = LOCAL_PROCESSED_DIR / f"thermal_{camera}"

        # Find the MP4 file in the RGB directory
        rgb_video_files = list(rgb_dir.glob("*.MP4")) + list(rgb_dir.glob("*.mp4"))
        print("files in rgb_dir:", rgb_video_files)
        if len(rgb_video_files) == 0:
            print(f"No MP4 file found in {rgb_dir}. Skipping camera {camera}.")
            continue
        elif len(rgb_video_files) > 1:
            print(f"Multiple MP4 files found in {rgb_dir}. Using the first one.")
        rgb_video_path = rgb_video_files[0]

        # Find the MP4 file in the thermal directory
        thermal_video_files = list(thermal_dir.glob("*.mp4")) + list(
            thermal_dir.glob("*.MP4")
        )
        print("files in thermal_dir:", thermal_video_files)
        if len(thermal_video_files) == 0:
            print(f"No MP4 file found in {thermal_dir}. Skipping camera {camera}.")
            continue
        elif len(thermal_video_files) > 1:
            print(f"Multiple MP4 files found in {thermal_dir}. Using the first one.")
        thermal_video_path = thermal_video_files[0]

        print(f"RGB video path: {rgb_video_path}")
        print(f"Thermal video path: {thermal_video_path}")

        output_dir_rgb = LOCAL_PROCESSED_DIR / "aligned_frames" / f"rgb_{camera}"
        output_dir_thm = LOCAL_PROCESSED_DIR / "aligned_frames" / f"thermal_{camera}"
        output_dir_rgb.mkdir(parents=True, exist_ok=True)
        output_dir_thm.mkdir(parents=True, exist_ok=True)

        # Align videos
        print(f"Aligning videos for camera {camera}...")

        align_videos(
            rgb_video_path,
            thermal_video_path,
            output_dir_rgb,
            output_dir_thm,
            frame_size=frame_size,
            max_frames=max_frames,
            warp_to=warp_to,
            rotation_angle=rotation_angle,
            skip_homography=skip_homography,
            skip_translation=skip_translation,
        )

#### Object detection and tracking

In [13]:
# download yolo weights
gcs_client.gcs.get_file(
    "roboflow_model/yolov11_weights.pt", LOCAL_DOWNLOAD_DIR / "yolo11_weights.pt"
)

In [14]:
# Detection and tracking
print("Running detection...")
for camera in camera_numbers:
    print(f"Running detection and tracking on: thermal_{camera}")

    # Define paths for the thermal video and model inference
    thermal_video_path = (
        LOCAL_PROCESSED_DIR
        / "aligned_frames"
        / f"thermal_{camera}"
        / "warped_thermal_adjusted.mp4"
    )
    if not thermal_video_path.exists():
        print(f"Thermal video not found for camera {camera}. Skipping...")
        continue

    rgb_video_path = (
        LOCAL_PROCESSED_DIR
        / "aligned_frames"
        / f"rgb_{camera}"
        / "cropped_rgb_adjusted.mp4"
    )

    # Run local_model_inference script
    try:
        detect_csv = (
            LOCAL_PROCESSED_DIR
            / "aligned_frames"
            / f"thermal_{camera}"
            / f"detections_{camera}.csv"
        )
        output_video_path = (
            LOCAL_PROCESSED_DIR
            / "aligned_frames"
            / f"thermal_{camera}"
            / "annotated_warped_thermal.mp4"
        )
        checkpoint_path = (
            LOCAL_DOWNLOAD_DIR / "yolo11_weights.pt"
        )  # Update with your model path

        infer_with_yolo(
            video_path=thermal_video_path,
            model_path=checkpoint_path,
            output_csv_path=detect_csv,
            show_window=not smoke_test,
            verbose=not smoke_test,
            max_frames=100 if smoke_test else None,
        )
        print(
            f"Object detection completed for camera {camera}. Results saved to {detect_csv}."
        )
    except Exception as e:
        print(f"Error during object detection for camera {camera}: {e}")
        continue

Running detection...
Running detection and tracking on: thermal_1
Model loaded successfully for inference!
Total frames in video: 100

0: 480x640 5 birds, 18.4ms
Speed: 0.0ms preprocess, 18.4ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 birds, 16.5ms
Speed: 0.0ms preprocess, 16.5ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 birds, 15.0ms
Speed: 0.0ms preprocess, 15.0ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 birds, 14.9ms
Speed: 0.0ms preprocess, 14.9ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 birds, 15.9ms
Speed: 0.0ms preprocess, 15.9ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 birds, 17.4ms
Speed: 0.0ms preprocess, 17.4ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 birds, 16.1ms
Speed: 0.0ms preprocess, 16.1ms inference, 0.4ms postprocess per image at shape (1, 3, 4

In [15]:
# running tracking
for camera in camera_numbers:

    # Define paths for the thermal video and model inference
    thermal_video_path = (
        LOCAL_PROCESSED_DIR
        / "aligned_frames"
        / f"thermal_{camera}"
        / "warped_thermal_adjusted.mp4"
    )
    if not thermal_video_path.exists():
        print(f"Thermal video not found for camera {camera}. Skipping...")
        continue

    rgb_video_path = (
        LOCAL_PROCESSED_DIR
        / "aligned_frames"
        / f"rgb_{camera}"
        / "cropped_rgb_adjusted.mp4"
    )
    if not rgb_video_path.exists():
        print(f"RGB video not found for camera {camera}. Skipping...")
        continue

    # Check if the detection CSV exists
    detect_csv = (
        LOCAL_PROCESSED_DIR
        / "aligned_frames"
        / f"thermal_{camera}"
        / f"detections_{camera}.csv"
    )
    if not detect_csv.exists():
        print(f"Detection CSV not found for camera {camera}. Skipping tracking.")
        continue
    # Visualize detections on the thermal and RGB videos
    # visualization
    get_detections_from_video(
        csv_path=detect_csv,
        video_path=thermal_video_path,
        output_video_path=LOCAL_PROCESSED_DIR
        / "aligned_frames"
        / f"thermal_{camera}"
        / f"visualized_thermal_{camera}.mp4",
    )
    # visualization
    get_detections_from_video(
        csv_path=detect_csv,
        video_path=rgb_video_path,
        output_video_path=LOCAL_PROCESSED_DIR
        / "aligned_frames"
        / f"rgb_{camera}"
        / f"visualized_rgb_{camera}.mp4",
    )
    # Run tracking
    print(f"Running tracking on: camera {camera}")
    run_tracking(LOCAL_PROCESSED_DIR / "aligned_frames", "thermal", camera)
    run_tracking(LOCAL_PROCESSED_DIR / "aligned_frames", "rgb", camera)
    tracked_csv = (
        LOCAL_PROCESSED_DIR
        / "aligned_frames"
        / f"thermal_{camera}"
        / f"thermal_{camera}_tracks.csv"
    )
    if not tracked_csv.exists():
        print(f"Tracking CSV not found for camera {camera}. Skipping visualization.")
        continue
    # Output tracked bounding boxes to CSV
    output_tracked_bboxes_csv(
        track_csv=tracked_csv,
        detect_csv=detect_csv,
        output_csv=Path(
            LOCAL_PROCESSED_DIR
            / f"aligned_frames/rgb_{camera}/tracked_bboxes_{camera}.csv"
        ),
        iou_threshold=0.1,
    )  # Lower threshold if tracks are not centered

    print(f"Visualizing tracks for rgb camera {camera}...")
    # TODO: this doesn't work
    # overlay_tracks_on_video(
    #     csv_path=tracked_csv,
    #     frame_dir=LOCAL_PROCESSED_DIR
    #     / "aligned_frames"
    #     / f"rgb_{camera}"
    #     / "annotated_frames",
    #     output_video=LOCAL_PROCESSED_DIR
    #     / "aligned_frames"
    #     / f"rgb_{camera}"
    #     / f"overlayed_tracks_{camera}.mp4",
    # )

Processing video: 100%|██████████| 964/964 [00:00<00:00, 1164.50it/s]


✅ Annotated video saved to: /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames/thermal_1/visualized_thermal_1.mp4


Processing video: 100%|██████████| 973/973 [00:01<00:00, 699.07it/s]


✅ Annotated video saved to: /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames/rgb_1/visualized_rgb_1.mp4
Running tracking on: camera 1
Running tracking for thermal camera 1 in session: /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames
✅ Tracking results saved to /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames/thermal_1/thermal_1_tracks.csv
Running tracking for rgb camera 1 in session: /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames
using detections from thermal camera
⚠️ No detections CSV found at: /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames/thermal_{camera_number}/detections_1.csv
✅ Tracked bounding box CSV saved to: /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames/rgb_1/tracked_bboxes_1.csv
Visualizing tracks for rgb camera 1...


### Optional: Visualization

In [16]:
frame_number = 100  # Example frame number to visualize
camera = 1  # Example camera number to visualize

bboxes_csv = (
    LOCAL_PROCESSED_DIR
    / "aligned_frames"
    / f"rgb_{camera}"
    / f"tracked_bboxes_{camera}.csv"
)

print(f"Plotting tracks at frame {frame_number} for camera {camera}...")
plot_tracks_at_frame_bbox_from_video(
    tracked_bboxes_csv=bboxes_csv,
    video_path=rgb_video_path,
    output_image=LOCAL_PROCESSED_DIR
    / "aligned_frames"
    / f"rgb_{camera}"
    / f"tracked_bboxes_{camera}_{frame_number}.png",
    frame_number=frame_number,
    max_frame=1000,
)

# If possible, generate pixel masks within each bounding box
print(f"Exporting tracks with masks for camera {camera}...")

generate_thermal_masks_from_bboxes(
    bbox_csv=bboxes_csv,
    video_path=LOCAL_PROCESSED_DIR
    / "aligned_frames"
    / f"thermal_{camera}"
    / "warped_thermal_adjusted.mp4",
    output_mask_dir=LOCAL_PROCESSED_DIR
    / "aligned_frames"
    / f"thermal_{camera}"
    / "masks",
    temp_threshold=128,  # default threshold for thermal images, half of 255
    mask_value=255,
)
export_tracks_with_masks(
    tracked_bboxes_csv=bboxes_csv,
    mask_dir=LOCAL_PROCESSED_DIR / "aligned_frames" / f"rgb_{camera}" / "masks",
    output_csv=LOCAL_PROCESSED_DIR
    / "aligned_frames"
    / f"rgb_{camera}"
    / f"tracked_bboxes_{camera}_with_masks.csv",
)

Plotting tracks at frame 100 for camera 1...
✅ Track plot saved to: /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames/rgb_1/tracked_bboxes_1_100.png
Exporting tracks with masks for camera 1...
✅ Thermal masks saved to: /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames/thermal_1/masks
✅ Exported tracks with masks to: /Users/dima/git/collab-environment/data/processed/test-session/aligned_frames/rgb_1/tracked_bboxes_1_with_masks.csv


### Optional: Push processed files

In [17]:
if not smoke_test:
    # Upload Processed Data to Cloud Bucket
    BUCKET_NAME = "fieldwork_processed"
    CLOUD_PROCESSED_PREFIX = (
        "your-cloud-processed-prefix"  # Update with your processed data prefix
    )
    for file in LOCAL_PROCESSED_DIR.iterdir():
        cloud_path = f"{BUCKET_NAME}/{CLOUD_PROCESSED_PREFIX}/{file.name}"
        gcs_client.upload_file(str(file), cloud_path)

    print("Uploaded processed files:", list(LOCAL_PROCESSED_DIR.iterdir()))